In [13]:
# ライブラリがなければインストール
# !pip install torch torch-geometric pandas numpy matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import pandas as pd

In [14]:
# import numpy as np
# import pandas as pd
# import networkx as nx
# import matplotlib.pyplot as plt
# from matplotlib.lines import Line2D

# # --- 1. & 2. & 3. データ準備（変更なし） ---
# np.random.seed(42)
# nodes = {
#     'Inf_A': {'type': 'Influencer'}, 'Inf_B': {'type': 'Influencer'},
#     '#fashion': {'type': 'Hashtag'}, '#food': {'type': 'Hashtag'},
#     '#travel': {'type': 'Hashtag'}, 'dress': {'type': 'Object'},
#     'pizza': {'type': 'Object'}, 'Brand_C': {'type': 'User'}
# }
# node_list = sorted(nodes.keys())
# node_to_id = {name: i for i, name in enumerate(node_list)}
# edges_t1 = [
#     ('Inf_A', '#fashion'), ('Inf_A', 'dress'), ('Inf_A', 'Brand_C'),
#     ('Inf_B', '#food'), ('Inf_B', 'pizza')
# ]
# num_nodes = len(node_list)
# A1 = np.zeros((num_nodes, num_nodes), dtype=int)
# for u, v in edges_t1:
#     u_id, v_id = node_to_id[u], node_to_id[v]
#     A1[u_id, v_id] = 1
#     A1[v_id, u_id] = 1
# A1_df = pd.DataFrame(A1, index=node_list, columns=node_list)
# feature_dim = 4
# X1 = np.random.rand(num_nodes, feature_dim).round(3)
# X1_df = pd.DataFrame(X1, index=node_list, columns=[f'feature_{i+1}' for i in range(feature_dim)])

# # --- 表示は省略 ---
# # print("--- Adjacency Matrix A_1 ---")
# # print(A1_df)
# # print("\n" + "--- Node Feature Matrix X_1 ---")
# # print(X1_df)


# # --- 4. グラフの可視化（修正箇所） ---

# G = nx.from_pandas_adjacency(A1_df)
# G.add_node('#travel')

# color_map = {
#     'Influencer': '#ff4757', # 赤
#     'Hashtag': '#2ed573',    # 緑
#     'Object': '#1e90ff',     # 青
#     'User': '#ffa502'        # オレンジ
# }
# node_colors = [color_map[nodes[node]['type']] for node in G.nodes()]

# # **修正点①:** ノード間の斥力（反発力）を強くして、レイアウト全体を広げます
# # kの値を大きくすると、ノード同士が離れます
# pos = nx.spring_layout(G, seed=42, k=1.5)

# fig, ax = plt.subplots(figsize=(14, 10)) # 図全体のサイズを大きくする

# # ノードを描画
# nx.draw_networkx_nodes(G, pos, ax=ax, node_color=node_colors, node_size=3000)

# # エッジ（線）を描画
# nx.draw_networkx_edges(G, pos, ax=ax, edge_color='gray', width=1.5, alpha=0.8)

# # **修正点②:** ラベルをノードの中心から少しずらして描画し、重なりを防ぐ
# # ラベルだけを別の関数で描画することで、位置の微調整が可能になります
# label_pos = {k: [v[0], v[1] + 0.08] for k, v in pos.items()} # Y座標を少し上にずらす
# nx.draw_networkx_labels(G, label_pos, ax=ax, font_size=12, font_weight='bold')


# # 凡例（レジェンド）を作成
# legend_elements = [Line2D([0], [0], marker='o', color='w', label=node_type,
#                           markerfacecolor=mfc, markersize=15)
#                    for node_type, mfc in color_map.items()]
# ax.legend(handles=legend_elements, title='Node Types', loc='upper right')
# ax.set_title('Visualization of Heterogeneous Graph G_1 (Improved Layout)', fontsize=16)

# # 枠線を消してスッキリさせる
# ax.margins(0.1)
# plt.axis('off')
# plt.show()

In [15]:
# --- データ定義 ---
node_list = [
    'Inf_A', 'Inf_B', '#fashion', '#food', '#travel',
    'dress', 'pizza', 'Brand_C'
]
node_to_id = {name: i for i, name in enumerate(node_list)}
num_nodes = len(node_list)

# t=1の時点での繋がり (エッジ)
edges_t1 = [
    ('Inf_A', '#fashion'), ('Inf_A', 'dress'), ('Inf_A', 'Brand_C'),
    ('Inf_B', '#food'), ('Inf_B', 'pizza')
]

# --- ① 隣接行列 A_1 (関係性の地図) の作成 ---
A1 = np.zeros((num_nodes, num_nodes), dtype=int)
for u, v in edges_t1:
    u_id, v_id = node_to_id[u], node_to_id[v]
    A1[u_id, v_id] = 1
    A1[v_id, u_id] = 1 # 無向グラフなので対称

print("--- Adjacency Matrix A_1 (Shape: {}) ---".format(A1.shape))
print(pd.DataFrame(A1, index=node_list, columns=node_list))


# --- ② ノード特徴量行列 X_1 (各ノードのプロフィール) の作成 ---
# 論文に従い、特徴量の次元数を128に設定
feature_dim = 128
# 本来は意味のあるベクトルだが、ここではランダムに生成
np.random.seed(42)
X1 = np.random.rand(num_nodes, feature_dim)

print("\n--- Node Feature Matrix X_1 (Shape: {}) ---".format(X1.shape))
print("各行が1つのノードの128次元の特徴ベクトルに対応します。")
print(pd.DataFrame(X1).head())

--- Adjacency Matrix A_1 (Shape: (8, 8)) ---
          Inf_A  Inf_B  #fashion  #food  #travel  dress  pizza  Brand_C
Inf_A         0      0         1      0        0      1      0        1
Inf_B         0      0         0      1        0      0      1        0
#fashion      1      0         0      0        0      0      0        0
#food         0      1         0      0        0      0      0        0
#travel       0      0         0      0        0      0      0        0
dress         1      0         0      0        0      0      0        0
pizza         0      1         0      0        0      0      0        0
Brand_C       1      0         0      0        0      0      0        0

--- Node Feature Matrix X_1 (Shape: (8, 128)) ---
各行が1つのノードの128次元の特徴ベクトルに対応します。
        0         1         2         3         4         5         6    \
0  0.374540  0.950714  0.731994  0.598658  0.156019  0.155995  0.058084   
1  0.006952  0.510747  0.417411  0.222108  0.119865  0.337615  0.942910   
2

In [16]:
# --- GCNモデルの定義 ---
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers=2):
        super().__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        # 2層のGCNConvを定義
        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.convs.append(GCNConv(hidden_channels, hidden_channels))

    def forward(self, x, edge_index):
        layer_outputs = [] # 各層の出力を保存するリスト
        # 1層目
        x = self.convs[0](x, edge_index)
        x = F.relu(x)
        layer_outputs.append(x)

        # 2層目
        x = self.convs[1](x, edge_index)
        x = F.relu(x)
        layer_outputs.append(x)

        # 論文の Rt = [F(1), F(2), ..., F(e)] に従って出力を連結
        final_representation = torch.cat(layer_outputs, dim=1)
        return final_representation

# --- GCNの実行 ---
# モデルを初期化（入力次元=128, 隠れ層次元=128）
model = GCN(in_channels=feature_dim, hidden_channels=feature_dim, num_layers=2)

# Numpy配列をPyTorchテンソルに変換
x_tensor = torch.tensor(X1, dtype=torch.float)
edge_index_tensor = torch.tensor(np.array(A1.nonzero()), dtype=torch.long)

# モデルにデータを通して、最終表現 R_1 を得る
with torch.no_grad(): # 学習はしないので勾配計算は不要
    R1 = model(x_tensor, edge_index_tensor)

In [17]:
print("\n--- GCN-Encoded Representation R_1 (Shape: {}) ---".format(R1.shape))
print(f"次元数: {R1.shape[1]} = (隠れ層次元: {feature_dim}) * (GCN層数: {model.num_layers})")
print("\n先頭5ノードの最終特徴量ベクトル（最初の10次元のみ表示）:")
print(pd.DataFrame(R1.numpy()).head().iloc[:, :10])


--- GCN-Encoded Representation R_1 (Shape: torch.Size([8, 256])) ---
次元数: 256 = (隠れ層次元: 128) * (GCN層数: 2)

先頭5ノードの最終特徴量ベクトル（最初の10次元のみ表示）:
          0    1         2         3         4         5    6         7  \
0  0.209163  0.0  0.740956  0.379701  1.647790  0.000000  0.0  0.000000   
1  0.000000  0.0  0.350634  0.000000  1.524265  0.000000  0.0  0.000000   
2  0.188515  0.0  0.377466  0.236195  1.198816  0.034942  0.0  0.000000   
3  0.000000  0.0  0.324192  0.000000  1.407608  0.000000  0.0  0.113022   
4  0.000000  0.0  0.000000  0.067069  1.584769  0.000000  0.0  0.000000   

          8         9  
0  1.067960  0.350682  
1  1.150376  0.771530  
2  0.631247  0.046924  
3  1.094700  0.333433  
4  0.953883  0.000000  
